In [9]:
import sys
import os
# Add 'src' directory to Python path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if src_path not in sys.path:
    sys.path.append(src_path)

In [10]:

from pyspark.sql import SparkSession
# from pyspark.sql import functions as F
from src import helper_functions as hf
# from src import schema as sc   # your custom schema function
# from src import datacleaning as dcp
# from src import hf_analysis as hf
# from src import franchise_analysis as franch
# from src import visualisation as visn
from pyspark.sql.types import *
from pyspark.sql.functions import * 
from src import schema as sc


In [ ]:
#Initialize Spark
spark = SparkSession.builder \
         .appName("streaming_data") \
         .config("spark.jars", "/Users/gyauk/codetools/javajdbc/postgresql-42.7.5.jar") \
         .getOrCreate()

# spark.stop()
# spark = SparkSession.builder.appName("streaming_data").getOrCreate()


''' Define struct type of columns in dataframe 
read the csv files 

'''

# df = spark.read.option("header", "true").option("inferSchema", "true").csv("path/to/file.csv")
csvDF = spark.readStream.option("sep", ",").schema(sc.schema_build()).csv("/Users/gyauk/github/labs/data")
csvDF.writeStream.format("console").option("truncate", "false").start()



# Load JSON data using the schema
# json_path = "/Users/gyauk/github/labs/Pyspark_IMBD_movie_analysis/data/raw/movies.json"
# df = spark.read.schema(hf.schema_build()).json(json_path)


25/04/29 15:22:32 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Can't assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at java.base/sun.nio.ch.Net.bind0(Native Method)
	at java.base/sun.nio.ch.Net.bind(Net.java:461)
	at java.base/sun.nio.ch.Net.bind(Net.java:453)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:227)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:141)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:562)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1334)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContext

25/04/30 09:24:50 WARN FileStreamSource: Listed 29 file(s) in 2543 ms
25/04/30 09:24:50 WARN FileStreamSource: Listed 29 file(s) in 2468 ms


In [4]:
csvDF

DataFrame[customer_id: string, product_id: string, product_category: string, payment_type: string, device_type: string, event_type: string, event_date: string]

In [5]:
cleaned_df= hf.convert_column_types(csvDF)

root
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- device_type: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_date: date (nullable = true)



In [6]:
cleaned_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- device_type: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_date: date (nullable = true)



In [7]:
spark.conf.get("spark.jars")


'file:///Users/gyauk/codetools/javajdbc/postgresql-42.7.5.jar'

In [ ]:
# Attach this function to your streaming query
query = cleaned_df.writeStream \
    .foreachBatch(hf.write_to_postgres) \
    .outputMode("append") \
    .start()

query.awaitTermination(timeout=None)

25/04/28 17:20:14 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/4r/wjm49txx4vj1j_kryh7ps6r00000gn/T/temporary-b242243a-f3b7-4457-8686-33f041d309d6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/28 17:20:14 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Batch 0 started writing to PostgreSQL...


-------------------------------------------
Batch: 0
-------------------------------------------


+-----------+----------+----------------+------------+-----------+----------+----------+
|customer_id|product_id|product_category|payment_type|device_type|event_type|event_date|
+-----------+----------+----------------+------------+-----------+----------+----------+
|CUST-14987 |Prod_13987|Books           |Visa Card   |Mobile     |Purchase  |30-06-2024|
|CUST-14988 |Prod_13988|Electronics     |Paypal      |Mobile     |Purchase  |28-09-2023|
|CUST-14989 |Prod_13989|Clothes         |Momo        |Desktop    |Purchase  |25-09-2022|
|CUST-14990 |Prod_13990|Furniture       |Visa Card   |Desktop    |Addtocart |17-11-2024|
|CUST-14991 |Prod_13991|Furniture       |Visa Card   |Desktop    |Addtocart |22-07-2023|
|CUST-14992 |Prod_13992|Electronics     |Momo        |Mobile     |Purchase  |16-04-2020|
|CUST-14993 |Prod_13993|Beauty          |Paypal      |Mobile     |Purchase  |14-12-2022|
|CUST-14994 |Prod_13994|Clothes         |Visa Card   |Mobile     |Addtocart |03-12-2021|
|CUST-14995 |Prod_139

Batch 0 successfully written to PostgreSQL!


-------------------------------------------
Batch: 1
-------------------------------------------
+-----------+----------+----------------+------------+-----------+----------+----------+
|customer_id|product_id|product_category|payment_type|device_type|event_type|event_date|
+-----------+----------+----------------+------------+-----------+----------+----------+
|CUST-23978 |Prod_22978|Accessories     |Paypal      |Mobile     |Purchase  |26-06-2021|
|CUST-23979 |Prod_22979|Hardware        |Momo        |Desktop    |Addtocart |24-09-2023|
|CUST-23980 |Prod_22980|Beauty          |Visa Card   |Desktop    |Addtocart |04-02-2023|
|CUST-23981 |Prod_22981|Furniture       |Visa Card   |Desktop    |Purchase  |06-06-2021|
|CUST-23982 |Prod_22982|Accessories     |Visa Card   |Mobile     |Purchase  |27-05-2023|
|CUST-23983 |Prod_22983|Books           |Paypal      |Desktop    |Purchase  |20-07-2023|
|CUST-23984 |Prod_22984|Hardware        |Visa Card   |Desktop    |Addtocart |14-03-2024|
|CUST-23985 |

Batch 1 successfully written to PostgreSQL!
-------------------------------------------
Batch: 2
-------------------------------------------
+-----------+----------+----------------+------------+-----------+----------+----------+
|customer_id|product_id|product_category|payment_type|device_type|event_type|event_date|
+-----------+----------+----------------+------------+-----------+----------+----------+
|CUST-25976 |Prod_24976|Electronics     |Momo        |Desktop    |Addtocart |09-11-2021|
|CUST-25977 |Prod_24977|Electronics     |Visa Card   |Mobile     |Addtocart |11-10-2022|
|CUST-25978 |Prod_24978|Accessories     |Paypal      |Desktop    |Purchase  |23-02-2021|
|CUST-25979 |Prod_24979|Accessories     |Visa Card   |Desktop    |Addtocart |02-09-2021|
|CUST-25980 |Prod_24980|Accessories     |Visa Card   |Desktop    |Purchase  |16-01-2024|
|CUST-25981 |Prod_24981|Clothes         |Paypal      |Desktop    |Purchase  |19-04-2023|
|CUST-25982 |Prod_24982|Beauty          |Visa Card   |Mobi

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/anaconda3/envs/vscodeenv/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/anaconda3/envs/vscodeenv/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/anaconda3/envs/vscodeenv/lib/python3.8/socket.py", line 681, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 